## Dolly Zoom: Compute focal length

### 1) Introduction

In this lab, we will implement Dolly Zoom effect used by film-makers to create a sensation of vertigo, a ``falling-away-from-oneself feeling''. This lab is fairly simple and is meant to introduce you to the concepts of projection and focal length. It keeps the size of an object of interests constant in the image, while making the foreground and background objects appear larger or smaller by adjusting focal length and moving the camera. You will simulate the Dolly Zoom effect with a synthetic scene as shown in next Figure, which illustrates two cubes and one pyramid seen from the top view.

<img src='../imgs/dolly_top_view.png' width="500"/>

### 2) Dolly Zoom Effect

A point in 3D is projected onto the image plane through the pinhole (center of projection, COP):

$$u = f\frac{X}{Z} \ \ \ \ \ \ \ \ \ \ \ v = f\frac{Y}{Z}$$

where $(u, v)$ is the image coordinate of the projection, $(X, Y, Z)$ is the 3D point, and  is the focal length of the camera. When the camera moves along with its Z-axis, the depth, Z, changes and therefore, the projection, $(u, v)$, changes. In our paticular case the $Z$ of interest is $d_{ref}$, the depth of the objects in the scene. In the following discussion we will only mention the $u$ coordinate to simplify the equations, as we are focused mainly on height for the dolly zoom. This projection change produced by the depth change can be compensated by adjusting focal length:

where $pos$ is the movement of the camera along its $Z$ axis ($+$ direction indicates approaching to objects) and $f'$ is the modified focal length. $f_{ref}$ and $d_{ref}$ are the focal length and depth of an object in the original image, respectively. Dolly zoom effect exploits the compensation between depth and focal length, which produces depth sensation. The relationship between all the variable names as given in the code is described in the next figure, and when implementing the description in the code you should reference it.

<img src='../imgs/dolly_relations.png' width="400"/>

Finally, the next figure illustrates the focal length/depth compensation: the camera moves away from the object while changing its focal length such that the height of the object A, $h_1 = 400$, in both original and moved images remains constant. Note that the heights of the other background objects are changed due to this effect.

<img src='../imgs/dolly_view.png' width="600"/>

### 3) Implementation

For the first part of this lab, you need to implement the function `compute_focal_length`. Given the depth of the object of interest $d_{ref}$, and the focal length $f_{ref}$ of the camera for the original image, you need to estimate the modified focal length $f'$, such that the height of the object remains constant as the camera moves in the -axis (different input $pos$ values). When you have implemented the function, you can demonstrate the dolly soom effect in the synthetic scene above by using the `run_dolly_zoom` script we provide in the zip file `Lab1.zip`.

In [8]:
import numpy as np

def compute_focal_length(d_ref, f_ref, pos):
    """
    Compute camera focal length using given camera position

    Input:
        - d_ref: 1 by 1 double, distance of the object whose size remains constant
        - f_ref: 1 by 1 double, previous camera focal length
        - pos: 1 by n, each element represent camera center position on the z axis.
    Output:
        - f: 1 by n, camera focal length
    """
    pos = np.array(pos)
    # get the current position of the camera in relation to the object
    pos = d_ref - pos
    f = (f_ref*pos)/d_ref
    return f

### Code to call your function

In [12]:
d_ref = 4
f_ref = 400
pos = -5
f = compute_focal_length( d_ref, f_ref, pos )
print f

900


## Dolly Zoom: Compute focal length and position

### Dolly Zoom: Compute focal length and position

For the second part of this lab, you need to implement the function `compute_f_pos`. This time you are given the distances of two objects ($d_{1\_ref}$ and $d_{2\_ref}$) and their corresponding heights in the physical world ($H_1$ and $H_2$ respectively), and for a specified original focal length $f_{ref}$, you need to estimate the modified focal length $f'$ such that the ratio of the heights of the two objects in the image $\frac{h_1}{h_2}$ is the same as the input value $ratio$, while the height $h_1$ of the first object remains constant. The description and the notation of the problem is the same as the one presented in the previous part, so you can use it as reference.


### Resolution of the problem

Calculate $h_1$ and $h_2$:

$$h_1 = f_{ref} * \frac{H_1}{d_{1\_ref}}$$

$$h_2 = f_{ref} * \frac{H_2}{d_{2\_ref}}$$

Knowing that: 

$$h_1' = f * \frac{H_1}{d_{1\_ref} - pos}$$

$$h_2' = f * \frac{H_2}{d_{2\_ref} - pos}$$

$$ratio = \frac{h_1'}{h_2'}$$

$$h_1' = h_1$$

Then:

$$ratio = \frac{f *H_1}{d_{1\_ref} - pos} \div \frac{f *H_2}{d_{2\_ref} - pos} $$

$$\frac{h_1'}{h_2'} = \frac{f * H_1}{d_{1\_ref} - pos} * \frac{d_{2\_ref} - pos}{f * H_2} $$

$$\frac{h_1'}{h_2'} = \frac{f * H_1 (d_{2\_ref} - pos)}{f * H_2 (d_{1\_ref} - pos)}$$

$$\frac{h_1'}{h_2'} = \frac{H_1 * d_{2\_ref} - H_1 * pos}{H_2 * d_{1\_ref} - H_2 * pos}$$

$$h_1' * (H_2 * d_{1\_ref} - H_2 * pos) = h_2' * (H_1 * d_{2\_ref} - H_1 * pos)$$

$$(h_1' * H_2 * d_{1\_ref}) - (h_1' * H_2 * pos) = (h_2' * H_1 * d_{2\_ref}) - (h_2' * H_1 * pos)$$

$$(h_2' * H_1 * pos) - (h_1' * H_2 * pos) = (h_2' * H_1 * d_{2\_ref}) - (h_1' * H_2 * d_{1\_ref})$$

$$(h_2' * H_1 - h_1' * H_2) * pos = (h_2' * H_1 * d_{2\_ref}) - (h_1' * H_2 * d_{1\_ref})$$

$$pos = \frac{h_2' * H_1 * d_{2\_ref} - h_1' * H_2 * d_{1\_ref}}{h_2' * H_1 - h_1' * H_2}$$

In order to calculate the camera local length:

$$h_1 = f * \frac{H_1}{d_{1\_ref} - pos}$$

$$f = \frac{h_1}{H_1} * (d_{1\_ref} - pos)$$

In [14]:
def compute_f_pos(d1_ref, d2_ref, H1, H2, ratio, f_ref):
    """
    Compute camera focal length and camera position to achieve centain ratio
    between objects
    
    Input:
        - d1_ref: distance of the first object
        - d2_ref: distance of the second object
        - H1: height of the first object in physical world
        - H2: height of the second object in physical world
        - ratio: ratio between two objects (h1/h2)
        - f_ref: 1 by 1 double, previous camera focal length
    Output:
        - f: 1 by 1, camera focal length
        - pos: 1 by 1, camera position
    """
    # calculate h1 and h2
    h1 = f_ref * float(H1)/d1_ref
    h2 = f_ref * float(H2)/d2_ref
    
    h1_n = h1
    h2_n = h1_n / float(ratio)
    
    # calculate the position
    pos = ((h2_n * H1 * d2_ref) - (h1_n * H2 * d1_ref))/((h2_n * H1) - (h1_n * H2))
    
    # calculate the focal length
    f = (h1 / H1) * (d1_ref - pos)
    
    return f, pos 

### Code to call your function

In [15]:
d1_ref = 4;
d2_ref = 20;
H1 = 4;
H2 = 6;
ratio = 2;
f_ref = 400;
f, pos = compute_f_pos( d1_ref, d2_ref, H1, H2, ratio, f_ref )
print f
print pos

800.0
-4.0


# TESTING FUNCTIONS

In [32]:
import scipy.io as sio
import numpy as np

metadata = sio.whosmat('../lab/asset_1/points.mat')

keys = []
for k in metadata:
    keys.append(k[0])
print 'Keys', keys

points = sio.loadmat('../lab/asset_1/points.mat')
for k in points:
    if k in keys:
        print 'Matrix: ', k
        print points[k]
        print
        
d_ref = 4
f_ref = 400
pos = np.arange(0,-10,-0.1)

d1_ref = 4
d2_ref = 20
H1 = points['points_A'][0][1] - points['points_A'][1][1]
H2 = points['points_C'][0][1] - points['points_C'][1][1]
ratio = 2

print 'H1:', H1
print 'H2:', H2

Keys ['points_A', 'points_B', 'points_C']
Matrix:  points_A
[[-6.  2.  4.]
 [-6. -2.  4.]
 [-4.  2.  3.]
 [-4. -2.  3.]
 [-2.  2.  4.]
 [-2. -2.  4.]]

Matrix:  points_B
[[ 1. -2.  4.]
 [ 2.  1.  3.]
 [ 2. -2.  2.]
 [ 6. -2.  4.]]

Matrix:  points_C
[[ -8.   3.  20.]
 [ -8.  -3.  20.]
 [ -4.   3.  20.]
 [ -4.  -3.  20.]
 [ -4.   3.  24.]
 [ -4.  -3.  24.]]

H1: 4.0
H2: 6.0


In [33]:
def project_objects(f, pos, points, fid):
    """
    Render synthetic image using given camera focal length and camera
    position
    
    Input:
        - f: double camera focal length
        - pos: double represent camera center position in z axis.
        - points: 3D coordinates for vetice on polygons (use "load points.mat" to get)
    Output:
        - img: 1080*1920*3 matrix, the output render image
    after runing Dolly_Zoom
    you can use 'imwrite(img, 'output.png');' to save the image.
    """
    #TODO: Transform to Python plots
    pass
    #     color_1 = [0 1 0];
    #     color_2 = [1 0 1];
    #     color_3 = [0 0 1];

    #     points_A = points.points_A;
    #     points_B = points.points_B;
    #     points_C = points.points_C;

    #     figure(fid);
    #     #object 3
    #     p2d = project(points_C, f, pos);
    #     color = color_3;
    #     fill(p2d([1,2,4,3],1), p2d([1,2,4,3],2), color);
    #     fill(p2d([3,4,6,5],1), p2d([3,4,6,5],2), color);

    #     #object 1
    #     p2d = project(points_A, f, pos);
    #     color = color_1;
    #     fill(p2d([1,2,4,3],1), p2d([1,2,4,3],2), color);
    #     fill(p2d([3,4,6,5],1), p2d([3,4,6,5],2), color);

    #     #object 2
    #     p2d = project(points_B, f, pos);
    #     color = color_2;
    #     fill(p2d([1,2,3],1), p2d([1,2,3],2), color);
    #     fill(p2d([2,3,4],1), p2d([2,3,4],2), color);


def project(p3d, f, pos):
    """
    Use for compute vertex image position from given vertex 3D position and
    camera focal length and camera position
    
    Input:
        - p3d: n by 3, 3D vertex position in world coordinate system
        - f: double, camera focal length
        - pos : double, camera center position
    Output:
        - p2d: n by 2, each row represents vertex image position, in pixel unit
    """
    #TODO: Transform to Python plots
    pass
    #     p2d(:,1) = p3d(:,1)*f./(p3d(:,3) - pos) + 960;
    #     p2d(:,2) = p3d(:,2)*f./(p3d(:,3) - pos) + 540;
    p2d = 0.0
    return p2d

In [ ]:
# Dolly Zoom: keep one object's height constant 
f = compute_focal_length(d_ref, f_ref, pos)
for i in range(1, len(f)):
    if i == 1:
        print('Processing frame %03d / %d...' % (i, len(f)))
    else
        #fprintf(repmat('\b',1,12));  
        print('%03d / %d...' % (i, len(f)))
        #clf;
    
    #figure(1), hold on, axis equal;
    #xlim([0,1920]), ylim([0,1080]);
    project_objects(f[i], pos[i], points, 1)
    #pause(0.1);

print('\n')

# Dolly Zoom:  keep one object's height constant and adjust another objects height

f, pos = compute_f_pos(d1_ref, d2_ref, H1, H2, ratio, f_ref)
#figure(2), hold on, axis equal;
#xlim([0,1920]), ylim([0,1080]);
project_objects(f, pos, points, 2)